# Digit Classification with Neural Networks 

Interest in neural networks, and in particular those with architechures that support deep learning, has been surging in recent years.  

In this notebook we will be revisiting the problem of digit classification on the MNIST data.  In doing so, we will introduce a new Python library, Theano, for working with neural networks.  Theano is a popular choice for neural networks as the same code can be run on either CPUs or GPUs.  GPUs greatly speed up the training and prediction, and is readily available. Amazon even offers GPU machines on EC2.  

In part 1, we'll introduce Theano, and refresh ourselves on the MNIST dataset.  In part 2, we'll create a multi-layer neural network with a simple architechure, and train it using backpropagation.  Part 3 will introduce the convolutional architechure, which can be said to be doing 'deep learning' (also called feature learning or representation learning).

#### Part 1: Basics

Lets start to look at Theano.  If later you'd like to go deeper into Theano, you may want to read this paper: http://www.iro.umontreal.ca/~lisa/pointeurs/theano_scipy2010.pdf

Install Theano if you haven't already.  Then let's load it, and set it to work with a CPU.  For reference, here is the Theano documentation: http://www.deeplearning.net/software/theano/library/

In [2]:
%matplotlib inline

import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import time

import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print(theano.config.device) # We're using CPUs (for now)
print(theano.config.floatX) # Should be 64 bit for CPUs

np.random.seed(0)

cpu
float64


Now for MNIST data...

In [3]:
# Repeating steps from Project 1 to prepare mnist dataset. 
mnist = fetch_mldata('MNIST original', data_home='~/datasets/mnist')
X, Y = mnist.data, mnist.target
X = X / 255.0
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]
numExamples = 2000
test_data, test_labels = X[70000-numExamples:], Y[70000-numExamples:]
train_data, train_labels = X[:numExamples], Y[:numExamples]
numFeatures = train_data[1].size
numTrainExamples = train_data.shape[0]
numTestExamples = test_data.shape[0]
print('Features = %d' %(numFeatures))
print('Train set = %d' %(numTrainExamples))
print('Test set = %d' %(numTestExamples))

Features = 784
Train set = 2000
Test set = 2000


Looking ahead to working with neural networks, let's prepare one additional variation of the label data.  Let's make these labels, rather than each being an integer value from 0-9, be a set of 10 binary values, one for each class.  This is sometimes called a 1-of-n encoding, and it makes working with Neural Networks easier, as there will be one output node for each class.

In [4]:
def binarizeY(data):
    binarized_data = np.zeros((data.size,10))
    for j in range(0,data.size):
        feature = data[j:j+1]
        i = feature.astype(np.int64)
        binarized_data[j,i]=1
    return binarized_data
train_labels_b = binarizeY(train_labels)
test_labels_b = binarizeY(test_labels)
numClasses = train_labels_b[1].size
print('Classes = %d' %(numClasses))

Classes = 10


Lets start with a KNN model to establish a baseline accuracy.

Exercise: You've seen a number of different classification algorithms (e.g. naive bayes, decision trees, random forests, logistic regression) at this point.  How does KNN scalability and accuracy with respect to the size of the training dataset compare to those other algorithms?  

In [4]:
neighbors = 1
knn = KNeighborsClassifier(neighbors)
# we'll be waiting quite a while if we use 60K examples, so let's cut it down.  You may want to run the full 60K on your own later to see what the accuracy is.
mini_train_data, mini_train_labels = X[:numExamples], Y[:numExamples] 
start_time = time.time()
knn.fit(mini_train_data, mini_train_labels)
print('Train time = %.2f' %(time.time() - start_time))
start_time = time.time()
accuracy = knn.score(test_data, test_labels)
print('Accuracy = %.4f' %(accuracy))
print('Prediction time = %.2f' %(time.time() - start_time))

Train time = 0.09
Accuracy = 0.9130
Prediction time = 5.13


Alright, now that we have a simple baseline, let's start working in Theano.  Before we jump to multi-layer neural networks though, let's train a logistic regression model to make certain we're using Theano correctly. 

Recall from Josh's regression lecture the four key components: (1) parameters, (2) model, (3) cost function, and (4) objective. 

In [17]:
## (1) Parameters 
# Initialize the weights to small, but non-zero, values.
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))
print(numFeatures, numClasses, w.get_value().shape)


784 10 (784, 10)


Two notes relevant at this point:

First, logistic regression can be thought of as a neural network with no hidden layers. The output values are just the dot product of the inputs and the edge weights.

Second, we have 10 classes. We can either train separate one vs all classifiers using sigmoid activation, which would be a hassle, or we can use the softmax activation, which is essentially a multi-class version of sigmoid. We'll use Theano's built-in implementation of softmax.

In [24]:
## (2) Model
# Theano objects accessed with standard Python variables
X = T.matrix()
Y = T.matrix()

def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

print(y_hat)

Softmax.0


We'll use cross-entropy as a cost function.  Cross entropy only considers the error between the true class and the prediction, and not the errors for the false classes.  This tends to cause the network to converge faster.  We'll use Theano's built-in cross entropy function.

In [25]:
## (3) Cost function
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))
print(cost)

mean


The objective is minimize the cost, and to do that we'll use batch gradient descent.

Exercise: What are the differences between batch, stochastic, and mini-batch gradient descent?  What are the implications of each for working on large datasets?

We'll use Theano's built-in gradient function.  Exercise: Do you recall from Josh's lecture what the gradient is for beta in logistic regression?

In [28]:
## (4) Objective (and solver)

alpha = 0.01
gradient = T.grad(cost=cost, wrt=w) 
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, 
                        allow_input_downcast=True) # computes cost, then runs update
y_pred = T.argmax(y_hat, axis=1) # select largest probability as prediction
predict = theano.function(inputs=[X], outputs=y_pred, 
                          allow_input_downcast=True)
def gradientDescent(epochs):
    trainTime = 0.0
    predictTime = 0.0
    for i in range(epochs):
        start_time = time.time()
        cost = train(train_data[0:len(train_data)], 
                     train_labels_b[0:len(train_data)])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, 
            np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescent(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.8725
2) accuracy = 0.8725
3) accuracy = 0.8725
4) accuracy = 0.8725
5) accuracy = 0.8725
6) accuracy = 0.8725
7) accuracy = 0.8725
8) accuracy = 0.8725
9) accuracy = 0.8725
10) accuracy = 0.8725
11) accuracy = 0.8725
12) accuracy = 0.8725
13) accuracy = 0.8725
14) accuracy = 0.8725
15) accuracy = 0.8725
16) accuracy = 0.8725
17) accuracy = 0.8725
18) accuracy = 0.8725
19) accuracy = 0.8725
20) accuracy = 0.8725
21) accuracy = 0.8725
22) accuracy = 0.8725
23) accuracy = 0.8725
24) accuracy = 0.8725
25) accuracy = 0.8725
26) accuracy = 0.8725
27) accuracy = 0.8725
28) accuracy = 0.8725
29) accuracy = 0.8725
30) accuracy = 0.8725
31) accuracy = 0.8725
32) accuracy = 0.8725
33) accuracy = 0.8725
34) accuracy = 0.8725
35) accuracy = 0.8725
36) accuracy = 0.8725
37) accuracy = 0.8725
38) accuracy = 0.8725
39) accuracy = 0.8725
40) accuracy = 0.8725
41) accuracy = 0.8725
42) accuracy = 0.8725
43) accuracy = 0.8725
44) accuracy = 0.8725
45) accuracy = 0.8725
46) accuracy = 0.87

Exercise:  What do you expect to happen if we convert batch gradient descent to stochastic gradient descent?  Why?

Let's try it...

In [29]:
## (1) Parameters
w = theano.shared(np.asarray(
        (np.random.randn(*(numFeatures, numClasses))*.01)))

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

## (4) Objective
alpha = 0.01
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, 
                        allow_input_downcast=True) 
y_pred = T.argmax(y_hat, axis=1) 
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):       
        for start, end in zip(range(0, len(train_data), miniBatchSize), 
                              range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, 
                np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))) )    
    print('train time = %.2f' %(trainTime))
    
gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.8445
2) accuracy = 0.8640
3) accuracy = 0.8700
4) accuracy = 0.8730
5) accuracy = 0.8720
6) accuracy = 0.8710
7) accuracy = 0.8720
8) accuracy = 0.8695
9) accuracy = 0.8705
10) accuracy = 0.8725
11) accuracy = 0.8715
12) accuracy = 0.8700
13) accuracy = 0.8705
14) accuracy = 0.8705
15) accuracy = 0.8705
16) accuracy = 0.8715
17) accuracy = 0.8730
18) accuracy = 0.8725
19) accuracy = 0.8730
20) accuracy = 0.8740
21) accuracy = 0.8735
22) accuracy = 0.8735
23) accuracy = 0.8735
24) accuracy = 0.8730
25) accuracy = 0.8735
26) accuracy = 0.8740
27) accuracy = 0.8745
28) accuracy = 0.8745
29) accuracy = 0.8740
30) accuracy = 0.8735
31) accuracy = 0.8730
32) accuracy = 0.8725
33) accuracy = 0.8730
34) accuracy = 0.8725
35) accuracy = 0.8730
36) accuracy = 0.8720
37) accuracy = 0.8730
38) accuracy = 0.8730
39) accuracy = 0.8740
40) accuracy = 0.8735
41) accuracy = 0.8740
42) accuracy = 0.8740
43) accuracy = 0.8740
44) accuracy = 0.8735
45) accuracy = 0.8735
46) accuracy = 0.87

Exercise: What do you expect to happen if you switch the batch size to be great than 1 (i.e. mini-batch)?  Why?

Try it for a few values...

In [30]:
## (1) Parameters
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

## (4) Objective
alpha = 0.01
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) 
y_pred = T.argmax(y_hat, axis=1) 
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 10 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):       
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))    
    print('train time = %.2f' %(trainTime))
    
gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.8045
2) accuracy = 0.8285
3) accuracy = 0.8450
4) accuracy = 0.8500
5) accuracy = 0.8580
6) accuracy = 0.8630
7) accuracy = 0.8655
8) accuracy = 0.8670
9) accuracy = 0.8690
10) accuracy = 0.8715
11) accuracy = 0.8725
12) accuracy = 0.8740
13) accuracy = 0.8750
14) accuracy = 0.8790
15) accuracy = 0.8790
16) accuracy = 0.8790
17) accuracy = 0.8800
18) accuracy = 0.8805
19) accuracy = 0.8805
20) accuracy = 0.8805
21) accuracy = 0.8805
22) accuracy = 0.8815
23) accuracy = 0.8815
24) accuracy = 0.8825
25) accuracy = 0.8835
26) accuracy = 0.8840
27) accuracy = 0.8850
28) accuracy = 0.8855
29) accuracy = 0.8850
30) accuracy = 0.8850
31) accuracy = 0.8850
32) accuracy = 0.8845
33) accuracy = 0.8860
34) accuracy = 0.8865
35) accuracy = 0.8870
36) accuracy = 0.8870
37) accuracy = 0.8870
38) accuracy = 0.8875
39) accuracy = 0.8890
40) accuracy = 0.8890
41) accuracy = 0.8890
42) accuracy = 0.8885
43) accuracy = 0.8885
44) accuracy = 0.8885
45) accuracy = 0.8880
46) accuracy = 0.88

#### PART 2: Multi-layer Neural Networks

As we start to get further into Neural Networks, if you'd like to take time on your own for an in-depth introduction on the state of the art in the topic, check out this excellent 1-day tutorial from KDD2014:

Part 1: http://videolectures.net/kdd2014_bengio_deep_learning/

Part 2: http://videolectures.net/tcmm2014_taylor_deep_learning/

---------

Let's take our implementation of logistic regression (which recall is in fact a single layer neural network), and add a hidden layer, making it a two layer neural network.  Because we have a hidden layer, we will now train the model using backpropagation.

Exercise: How do you expect this model to compare to KNN and logistic regression in terms of train time and accuracy?  Why?

Let's try it out...

In [7]:
## (1) Parameters
numHiddenNodes = 60 # 600 
w_1 = theano.shared(np.asarray(
        (np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray(
        (np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2]


## (2) Model
X = T.matrix()
Y = T.matrix()

#theano.printing.debugprint(Y) 
# Two notes:
# First, feed forward is the composition of layers (dot product + activation function)
# Second, activation on the hidden layer still uses sigmoid
def model(X, w_1, w_2):
    return T.nnet.softmax(T.dot(T.nnet.sigmoid(T.dot(X, w_1)), w_2))
y_hat = model(X, w_1, w_2)


## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.1
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, 
                        allow_input_downcast=True)
y_pred = T.argmax(y_hat, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, 
                          allow_input_downcast=True)
miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), 
                              range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, 
                np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.8050
2) accuracy = 0.8495
3) accuracy = 0.8755
4) accuracy = 0.8900
5) accuracy = 0.8970
6) accuracy = 0.8980
7) accuracy = 0.8985
8) accuracy = 0.9025
9) accuracy = 0.9000
10) accuracy = 0.9010
11) accuracy = 0.9000
12) accuracy = 0.9010
13) accuracy = 0.9010
14) accuracy = 0.9030
15) accuracy = 0.9035
16) accuracy = 0.9045
17) accuracy = 0.9050
18) accuracy = 0.9055
19) accuracy = 0.9060
20) accuracy = 0.9055
21) accuracy = 0.9055
22) accuracy = 0.9055
23) accuracy = 0.9060
24) accuracy = 0.9065
25) accuracy = 0.9065
26) accuracy = 0.9070
27) accuracy = 0.9070
28) accuracy = 0.9070
29) accuracy = 0.9075
30) accuracy = 0.9075
31) accuracy = 0.9075
32) accuracy = 0.9075
33) accuracy = 0.9070
34) accuracy = 0.9075
35) accuracy = 0.9075
36) accuracy = 0.9075
37) accuracy = 0.9075
38) accuracy = 0.9075
39) accuracy = 0.9075
40) accuracy = 0.9085
41) accuracy = 0.9080
42) accuracy = 0.9080
43) accuracy = 0.9080
44) accuracy = 0.9080
45) accuracy = 0.9075
46) accuracy = 0.90

Exercise: Change the number of nodes in the hidden layer?  What do you expect the impact to be?  What is the impact?

Answer: More nodes in hidden layer = slower and less accurate! But I thought it would be faster

--------

As interest in networks with more layers and more complicated architechures has increased, a couple of tricks have emerged and become standard practice.  Let's look at two of those--rectifier activation and dropout noise.

Exercise:  We saw an improvement from adding a hidden layer.  What do you expect to happen if a second hidden layer was added?  

Let's try it...

In [56]:
## (1) Parms
numHiddenNodes = 600 
w_1 = theano.shared(np.asarray(
        (np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray(
        (np.random.randn(*(numHiddenNodes, numHiddenNodes))*.01)))
w_3 = theano.shared(np.asarray(
        (np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2, w_3]

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w_1, w_2, w_3):
    return T.nnet.softmax(T.dot(T.nnet.sigmoid(T.dot(
                    T.nnet.sigmoid(T.dot(X, w_1)), w_2)), w_3))
y_hat = model(X, w_1, w_2, w_3)


## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.1
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, 
                        allow_input_downcast=True)
y_pred = T.argmax(y_hat, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, 
                          allow_input_downcast=True)

miniBatchSize = 10
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), 
                    range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, 
            np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.1045
2) accuracy = 0.1045
3) accuracy = 0.1045
4) accuracy = 0.1045
5) accuracy = 0.1045
6) accuracy = 0.1045
7) accuracy = 0.1045
8) accuracy = 0.1045
9) accuracy = 0.1045
10) accuracy = 0.1255
11) accuracy = 0.2090
12) accuracy = 0.2600
13) accuracy = 0.2740
14) accuracy = 0.2810
15) accuracy = 0.2870
16) accuracy = 0.2915
17) accuracy = 0.2995
18) accuracy = 0.3055
19) accuracy = 0.3130
20) accuracy = 0.3160
21) accuracy = 0.3220
22) accuracy = 0.3405
23) accuracy = 0.3610
24) accuracy = 0.3915
25) accuracy = 0.4580
26) accuracy = 0.5195
27) accuracy = 0.5530
28) accuracy = 0.5955
29) accuracy = 0.6470
30) accuracy = 0.7015
31) accuracy = 0.7385
32) accuracy = 0.7580
33) accuracy = 0.7725
34) accuracy = 0.7875
35) accuracy = 0.7910
36) accuracy = 0.7960
37) accuracy = 0.8045
38) accuracy = 0.8135
39) accuracy = 0.8205
40) accuracy = 0.8235
41) accuracy = 0.8295
42) accuracy = 0.8355
43) accuracy = 0.8385
44) accuracy = 0.8420
45) accuracy = 0.8475
46) accuracy = 0.84

#### Activation Revisted

Let's look at a recent idea around activation closely associated with deep learning.  In 2010, in a paper published at NIPS (https://www.utc.fr/~bordesan/dokuwiki/_media/en/glorot10nipsworkshop.pdf), Yoshua Bengio showed that rectifier activation works better empirically than sigmoid activation when used in the hidden layers.  

The rectifier activation is simple: f(x)=max(0,x).  Intuitively, the difference is that as a sigmoid activated node approaches 1 it stops learning even if error continues to be propagated to it, whereas the rectifier activated node continue to learn (at least in the positive direction).  It is not completely understood (per Yoshua Bengio) why this helps, but there are some theories being explored including as related to the benefits of sparse representations in networks. (http://www.iro.umontreal.ca/~bengioy/talks/KDD2014-tutorial.pdf).  Rectifiers also speed up training.

Although the paper was published in 2010, the technique didn't gain widespread adoption until 2012 when members of Hinton's group spread the word, including with this Kaggle entry: http://blog.kaggle.com/2012/11/01/deep-learning-how-i-did-it-merck-1st-place-interview/

Let's change the activation in our 2 layer network to rectifier and see what happens...

In [5]:
## (1) Parms
numHiddenNodes = 60 # 600 too slow
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2]


## (2) Model
X = T.matrix()
Y = T.matrix()

def model(X, w_1, w_2):
    return T.nnet.softmax(T.dot(T.maximum(T.dot(X, w_1), 0.), w_2))
y_hat = model(X, w_1, w_2)


## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, 
                        allow_input_downcast=True)
y_pred = T.argmax(y_hat, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 10
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), 
                range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, 
            np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.4920
2) accuracy = 0.5070
3) accuracy = 0.6060
4) accuracy = 0.6695
5) accuracy = 0.7680
6) accuracy = 0.8055
7) accuracy = 0.8230
8) accuracy = 0.8365
9) accuracy = 0.8505
10) accuracy = 0.8630
11) accuracy = 0.8735
12) accuracy = 0.8755
13) accuracy = 0.8790
14) accuracy = 0.8835
15) accuracy = 0.8820
16) accuracy = 0.8820
17) accuracy = 0.8800
18) accuracy = 0.8835
19) accuracy = 0.8845
20) accuracy = 0.8855
21) accuracy = 0.8860
22) accuracy = 0.8880
23) accuracy = 0.8865
24) accuracy = 0.8875
25) accuracy = 0.8870
26) accuracy = 0.8870
27) accuracy = 0.8870
28) accuracy = 0.8885
29) accuracy = 0.8890
30) accuracy = 0.8885
31) accuracy = 0.8885
32) accuracy = 0.8875
33) accuracy = 0.8880
34) accuracy = 0.8880
35) accuracy = 0.8880
36) accuracy = 0.8880
37) accuracy = 0.8875
38) accuracy = 0.8875
39) accuracy = 0.8875
40) accuracy = 0.8880
41) accuracy = 0.8885
42) accuracy = 0.8880
43) accuracy = 0.8890
44) accuracy = 0.8875
45) accuracy = 0.8890
46) accuracy = 0.88

#### Maxout Activation

So rectifier activation worked great!  

Exercise: try another type of activation called Maxout (or Max Pooling) activiation.  Maxout activation just selects the max input as the output.  Maxout is a type of pooling, a technique which performs particularly well for vision problems. For more background see: http://jmlr.org/proceedings/papers/v28/goodfellow13.pdf and http://www.quora.com/What-is-impact-of-different-pooling-methods-in-convolutional-neural-networks  

#### Noise

Previously when working with the MNIST data we saw a benefit in generalization from adding noise to the training data.  Let's try that again here, however this time with a trick for adding noise called 'Dropouts'.  The idea with dropouts is that instead of (or in addition to) adding noise to our inputs, we add noise by having each node return 0 with a certain probability during training.  This trick both improves generalization in large networks and speeds up training.

Hinton introduced the idea in 2012 and gave an explanation of why it's similar to bagging (http://arxiv.org/pdf/1207.0580v1.pdf)

Let's give it a try...

In [8]:
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

## (1) Parms
numHiddenNodes = 60 # 600 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, 
                                                  numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, 
                                                  numClasses))*.01)))
params = [w_1, w_2]


## (2) Model
X = T.matrix()
Y = T.matrix()
srng = RandomStreams()
def dropout(X, p=0.):
    if p > 0:
        X *= srng.binomial(X.shape, p=1 - p)
        X /= 1 - p
    return X

def model(X, w_1, w_2, p_1, p_2):
    return T.nnet.softmax(T.dot(dropout(T.maximum(T.dot(
                        dropout(X, p_1), w_1),0.), p_2), w_2))
y_hat_train = model(X, w_1, w_2, 0.2, 0.5)
y_hat_predict = model(X, w_1, w_2, 0., 0.)

## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat_train, Y))

## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, 
                        updates=update, allow_input_downcast=True)
y_pred = T.argmax(y_hat_predict, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, 
                          allow_input_downcast=True)
miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), 
                        range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.7615
2) accuracy = 0.8180
3) accuracy = 0.8755
4) accuracy = 0.8735
5) accuracy = 0.8880
6) accuracy = 0.8830
7) accuracy = 0.8865
8) accuracy = 0.8875
9) accuracy = 0.8885
10) accuracy = 0.8805
11) accuracy = 0.8995
12) accuracy = 0.9000
13) accuracy = 0.9000
14) accuracy = 0.9075
15) accuracy = 0.9020
16) accuracy = 0.9065
17) accuracy = 0.9040
18) accuracy = 0.8995
19) accuracy = 0.9100
20) accuracy = 0.9055
21) accuracy = 0.9120
22) accuracy = 0.9140
23) accuracy = 0.9085
24) accuracy = 0.9115
25) accuracy = 0.9120
26) accuracy = 0.9115
27) accuracy = 0.9140
28) accuracy = 0.9170
29) accuracy = 0.9105
30) accuracy = 0.9135
31) accuracy = 0.9130
32) accuracy = 0.9145
33) accuracy = 0.9195
34) accuracy = 0.9090
35) accuracy = 0.9110
36) accuracy = 0.9160
37) accuracy = 0.9145
38) accuracy = 0.9200
39) accuracy = 0.9230
40) accuracy = 0.9135
41) accuracy = 0.9170
42) accuracy = 0.9140
43) accuracy = 0.9150
44) accuracy = 0.9235
45) accuracy = 0.9190
46) accuracy = 0.91

Let's try once again adding a second hidden layer...

In [10]:
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

## (1) Parmeters
numHiddenNodes = 100 # 600 
w_1 = theano.shared(np.asarray((np.random.randn(*(numFeatures, numHiddenNodes))*.01)))
w_2 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numHiddenNodes))*.01)))
w_3 = theano.shared(np.asarray((np.random.randn(*(numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2, w_3]


## (2) Model
X = T.matrix()
Y = T.matrix()

srng = RandomStreams()
def dropout(X, p=0.):
    if p > 0:
        X *= srng.binomial(X.shape, p=1 - p)
        X /= 1 - p
    return X

def model(X, w_1, w_2, w_3, p_1, p_2, p_3):
    return T.nnet.softmax(T.dot(dropout(T.maximum(
            T.dot(dropout(T.maximum(
            T.dot(dropout(X, p_1), w_1),0.), p_2), w_2),0.), p_3), w_3))

y_hat_train = model(X, w_1, w_2, w_3, 0.2, 0.5,0.5)
y_hat_predict = model(X, w_1, w_2, w_3, 0., 0.,0.)

## (3) Cost...same as logistic regression
cost = T.mean(T.nnet.categorical_crossentropy(y_hat_train, Y))


## (4) Minimization.  Update rule changes to backpropagation.
alpha = 0.01
def backprop(cost, w):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        updates.append([w1, w1 - grad * alpha])
    return updates
update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, 
                        allow_input_downcast=True)
y_pred = T.argmax(y_hat_predict, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), 
                range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.2235
2) accuracy = 0.2760
3) accuracy = 0.7205
4) accuracy = 0.8275
5) accuracy = 0.8195
6) accuracy = 0.8690
7) accuracy = 0.8700
8) accuracy = 0.8765
9) accuracy = 0.8825
10) accuracy = 0.8905
11) accuracy = 0.8955
12) accuracy = 0.8905
13) accuracy = 0.8995
14) accuracy = 0.8950
15) accuracy = 0.8945
16) accuracy = 0.9110
17) accuracy = 0.9035
18) accuracy = 0.8995
19) accuracy = 0.9090
20) accuracy = 0.9055
21) accuracy = 0.9110
22) accuracy = 0.9155
23) accuracy = 0.9050
24) accuracy = 0.9185
25) accuracy = 0.9095
26) accuracy = 0.9080
27) accuracy = 0.9130
28) accuracy = 0.9105
29) accuracy = 0.9155
30) accuracy = 0.9040
31) accuracy = 0.9025
32) accuracy = 0.9140
33) accuracy = 0.9195
34) accuracy = 0.9105
35) accuracy = 0.9050
36) accuracy = 0.9120
37) accuracy = 0.9155
38) accuracy = 0.9220
39) accuracy = 0.9170
40) accuracy = 0.9105
41) accuracy = 0.9160
42) accuracy = 0.9075
43) accuracy = 0.9215
44) accuracy = 0.9085
45) accuracy = 0.9175
46) accuracy = 0.92

#### Part 3: Convolutional Neural Nets

Today, when the phrase 'deep learning' is used to describe a system, it is often a convolution neural network (or convonet).  The convonet architechture was largely developed in the late 90's at Bell Labs, but only very recently popularized.  It was developed for image recognition, and is described and implemented with 2d representations in mind.

Geoffrey Hinton has an excellent two-part lecture on the topic:

https://www.youtube.com/watch?v=6oD3t6u5EPs

https://www.youtube.com/watch?v=fueIAeAsGzA

Also, this code was partly taken from these tutorials, which are worth referring back to:

http://deeplearning.net/tutorial/lenet.html

http://ufldl.stanford.edu/tutorial/supervised/FeatureExtractionUsingConvolution/

https://www.youtube.com/watch?v=S75EdAcXHKk

http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/

In [26]:
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
from theano.tensor.nnet.conv import conv2d
from theano.tensor.signal.downsample import max_pool_2d

## (1) Parameters
numHiddenNodes = 600 #600 
patchWidth = 3
patchHeight = 3
featureMapsLayer1 = 32
featureMapsLayer2 = 64
featureMapsLayer3 = 128

# For convonets, we will work in 2d rather than 1d.  The MNIST images are 28x28 in 2d.
imageWidth = 28
train_data = train_data.reshape(-1, 1, imageWidth, imageWidth)
test_data = test_data.reshape(-1, 1, imageWidth, imageWidth)

# Convolution layers.  
w_1 = theano.shared(np.asarray((
            np.random.randn(*(featureMapsLayer1, 1, patchWidth, patchHeight))*.01)))
w_2 = theano.shared(np.asarray((
            np.random.randn(*(featureMapsLayer2, featureMapsLayer1, 
                              patchWidth, patchHeight))*.01)))
w_3 = theano.shared(np.asarray((
            np.random.randn(*(featureMapsLayer3, featureMapsLayer2, 
                              patchWidth, patchHeight))*.01)))

# Fully connected NN. 
w_4 = theano.shared(np.asarray((np.random.randn(*(
                    featureMapsLayer3 * 3 * 3, numHiddenNodes))*.01)))
w_5 = theano.shared(np.asarray((np.random.randn(*(
                    numHiddenNodes, numClasses))*.01)))
params = [w_1, w_2, w_3, w_4, w_5]

## (2) Model
X = T.tensor4() # conv2d works with tensor4 type
Y = T.matrix()

srng = RandomStreams()
def dropout(X, p=0.):
    if p > 0:
        X *= srng.binomial(X.shape, p=1 - p)
        X /= 1 - p
    return X

# Theano provides built-in support for add convolutional layers
def model(X, w_1, w_2, w_3, w_4, w_5, p_1, p_2):
    l1 = dropout(max_pool_2d(T.maximum(
                conv2d(X, w_1, border_mode='full'),0.), (2, 2)), p_1)
    l2 = dropout(max_pool_2d(T.maximum(
                conv2d(l1, w_2), 0.), (2, 2)), p_1)
    l3 = dropout(T.flatten(max_pool_2d(T.maximum(
                conv2d(l2, w_3), 0.), (2, 2)), outdim=2), p_1) # flatten to switch back to 1d layers
    l4 = dropout(T.maximum(T.dot(l3, w_4), 0.), p_2)
    return T.nnet.softmax(T.dot(l4, w_5))

y_hat_train = model(X, w_1, w_2, w_3, w_4, w_5, 0.2, 0.5)
y_hat_predict = model(X, w_1, w_2, w_3, w_4, w_5, 0., 0.)
y_x = T.argmax(y_hat, axis=1)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat_train, Y))

## (4) Minimization.  
def backprop(cost, w, alpha=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=w)
    updates = []
    for w1, grad in zip(w, grads):
        
        # adding gradient scaling
        acc = theano.shared(w1.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * grad ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        grad = grad / gradient_scaling
        updates.append((acc, acc_new))
        
        updates.append((w1, w1 - grad * alpha))
    return updates

update = backprop(cost, params)
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, 
                        allow_input_downcast=True)
y_pred = T.argmax(y_hat_predict, axis=1)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):
        for start, end in zip(range(0, len(train_data), miniBatchSize), 
                    range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print('train time = %.2f' %(trainTime))

gradientDescentStochastic(10)

start_time = time.time()
predict(test_data)   
print('predict time = %.2f' %(time.time() - start_time))

/Users/thongbui/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:49: UserWarning: pool_2d() will have the parameter ignore_border default value changed to True (currently False). To have consistent behavior with all Theano version, explicitly add the parameter ignore_border=True. On the GPU, using ignore_border=True is needed to use cuDNN. When using ignore_border=False and not using cuDNN, the only GPU combination supported is when `ds == st and padding == (0, 0) and mode == 'max'`. Otherwise, the convolution will be executed on CPU.
/Users/thongbui/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:51: UserWarning: pool_2d() will have the parameter ignore_border default value changed to True (currently False). To have consistent behavior with all Theano version, explicitly add the parameter ignore_border=True. On the GPU, using ignore_border=True is needed to use cuDNN. When using ignore_border=False and not using cuDNN, the only GPU combination supported is when 

1) accuracy = 0.8340
2) accuracy = 0.9190
3) accuracy = 0.9390
4) accuracy = 0.9430
5) accuracy = 0.9380
6) accuracy = 0.9530
7) accuracy = 0.9550
8) accuracy = 0.9585
9) accuracy = 0.9565
10) accuracy = 0.9560
train time = 4646.26
predict time = 9.46


#### Additional topic: Brain inspiration

The architechture of the convonet was inspired by the visual cortext in the human brain.  If you are interested in learning more, check out: http://www-psych.stanford.edu/~ashas/Cognition%20Textbook/chapter2.pdf

#### Additional topic: Self-Driving Vehicles

Convolutional networks are starting to be used more and more for self-driving vehicles.  This past year, NVIDEA started produced a chipset for self-driving vehicles that analyzes the video of up to 14 cameras using convonets for 

CES 2015 parts 6,7,9

https://www.youtube.com/watch?v=-vKGkxeflGw

https://www.youtube.com/watch?v=zsVsUvx8ieo

https://www.youtube.com/watch?v=RvQVyGOynFY

GTC 2015 parts 4,5,7,9

https://www.youtube.com/watch?v=pqvdZ2jp1NA

https://www.youtube.com/watch?v=GGxdP_JWhwI

https://www.youtube.com/watch?v=Tb7ZYSTYHbw

https://www.youtube.com/watch?v=TDm6Snkle70